<center><p float="center">
    <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<center><font size=10>Artificial Intelligence and Machine Learning</font></center>

<center><img src="https://img.freepik.com/free-photo/luxury-bedroom-suite-resort-high-rise-hotel-with-working-table_105762-1783.jpg?w=740&t=st=1654505493~exp=1654506093~hmac=fc1167df2b3797e32400448b458fd80afb85e31403c7689e0b1506023ff754a7" width="720"></center>



<center><font size=6>Hotel Booking Cancellation Prediction</font></center>

## Problem Statement

### Context

Many hotel bookings are called off due to cancellations or no-shows. The typical reasons for cancellations include change of plans, scheduling conflicts, etc. This is often made easier by the option to do so free of charge or preferably at a low cost, which benefits hotel guests. Still, it is a less desirable and possibly revenue-diminishing factor for hotels to deal with. Such losses are particularly high on last-minute cancellations.

The new technologies involving online booking channels have dramatically changed customers’ booking possibilities and behavior. This adds a further dimension to the challenge of how hotels handle cancellations, which are no longer limited to traditional booking and guest characteristics.


The cancellation of bookings impacts a hotel on various fronts:
1. Loss of resources (revenue) when the hotel cannot resell the room.
2. Additional costs of distribution channels by increasing commissions or paying for publicity to help sell these rooms.
3. Lowering prices last minute, so the hotel can resell a room, resulting in reducing the profit margin.
4. Human resources to make arrangements for the guests.

### Objective

The increasing number of cancellations calls for a Machine Learning based solution that can help in predicting which booking is likely to be canceled. INN Hotels Group has a chain of hotels in Portugal, they are facing problems with the high number of booking cancellations and have reached out to your firm for data-driven solutions. You as a data scientist have to analyze the data provided to find which factors have a high influence on booking cancellations, build a predictive model that can predict which booking is going to be canceled in advance, and help in formulating profitable policies for cancellations and refunds.


### Data Description

The data contains the different attributes of customers' booking details. The detailed data dictionary is given below.


**Data Dictionary**

* Booking_ID: unique identifier of each booking
* no_of_adults: Number of adults
* no_of_children: Number of Children
* no_of_weekend_nights: Number of weekend nights (Saturday or Sunday) the guest stayed or booked to stay at the hotel
* no_of_week_nights: Number of week nights (Monday to Friday) the guest stayed or booked to stay at the hotel
* type_of_meal_plan: Type of meal plan booked by the customer:
    * Not Selected – No meal plan selected
    * Meal Plan 1 – Breakfast
    * Meal Plan 2 – Half board (breakfast and one other meal)
    * Meal Plan 3 – Full board (breakfast, lunch, and dinner)
* required_car_parking_space: Does the customer require a car parking space? (0 - No, 1- Yes)
* room_type_reserved: Type of room reserved by the customer. The values are ciphered (encoded) by INN Hotels.
* lead_time: Number of days between the date of booking and the arrival date
* arrival_year: Year of arrival date
* arrival_month: Month of arrival date
* arrival_date: Date of the month
* market_segment_type: Market segment designation.
* repeated_guest: Is the customer a repeated guest? (0 - No, 1- Yes)
* no_of_previous_cancellations: Number of previous bookings that were canceled by the customer prior to the current booking
* no_of_previous_bookings_not_canceled: Number of previous bookings not canceled by the customer prior to the current booking
* avg_price_per_room: Average price per day of the reservation; prices of the rooms are dynamic. (in euros)
* no_of_special_requests: Total number of special requests made by the customer (e.g. high floor, view from the room, etc)
* booking_status: Flag indicating if the booking was canceled or not.

## Importing the necessary libraries

In [ ]:
# Code block for analysis/modeling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, recall_score, precision_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

## Loading the dataset

In [ ]:
# Code block for analysis/modeling
hotel = pd.read_csv('INNHotelsGroup.csv')
hotel.head()

## Data Overview

* Observations
* Sanity checks

In [ ]:
# Code block for analysis/modeling
print(hotel.info())
print('--' * 20)
print(hotel.describe().T)
print('--' * 20)
print(hotel.isnull().sum())
print('--' * 20)
print('Duplicate entries:', hotel.duplicated().sum())

## <a name='link2'>Exploratory Data Analysis (EDA) Summary</a>


- EDA is an important part of any project involving data.
- It is important to investigate and understand the data better before building a model with it.
- A few questions have been mentioned below which will help you approach the analysis in the right manner and generate insights from the data.
- A thorough analysis of the data, in addition to the questions mentioned below, should be done.

### EDA Observations:
1. **Busiest Months**: October is the busiest month, followed by September. The number of bookings peaks during the autumn season.
2. **Market Segments**: Online bookings constitute about 64% of the total, making it the primary revenue source.
3. **Pricing**: Aviation and Online segments have the highest average room prices, while Complementary bookings are (as expected) near zero.
4. **Cancellation Rate**: Approximately 32.7% of all bookings are canceled.
5. **Special Requests**: Bookings with more special requests are significantly less likely to cancel. 100% of bookings with 3 or more requests were completed.
6. **Repeated Guests**: Loyalty is a strong predictor; only ~1.7% of repeated guests cancel their stay.

### My Findings from the Exploratory Data Analysis:

1. **When is the hotel busiest?**
   - October is by far the busiest month with over 5,300 bookings. September and August are also quite packed, which suggests the late summer and autumn seasons are our peak times.

2. **Where do our guests come from?**
   - Most of our guests (about 64%) book through **Online** channels. Only about 29% are 'Offline' bookings.

3. **How do prices change across market segments?**
   - Online and Aviation segments are the most expensive (around 112€ and 100€ on average). Corporate and Offline bookings are more mid-range, while 'Complementary' bookings are near zero, as expected.

4. **What's our overall cancellation rate?**
   - Looking at the big picture, about **32.8%** of all bookings in this dataset ended up being canceled. That's nearly 1 in 3 bookings!

5. **Do our regular guests cancel often?**
   - Not at all! Only **1.7%** of repeating guests cancel. It turns out that once someone stays with us once, they are incredibly likely to show up for their next visit.

6. **Do special requests help reduce cancellations?**
   - Absolutely. There's a clear trend: the more special requests a guest makes, the less likely they are to cancel. In fact, guests with 3 or more requests had a 0% cancellation rate in our data!

In [ ]:
# Code to find the answers to EDA questions

print('1. Busiest Months (Top 5):')
display(hotel['arrival_month'].value_counts().head())

print('\n2. Market Segment Distribution (%):')
display(hotel['market_segment_type'].value_counts(normalize=True) * 100)

print('\n3. Average Price per Room by Market Segment:')
display(hotel.groupby('market_segment_type')['avg_price_per_room'].mean().sort_values(ascending=False))

print('\n4. Overall Cancellation Rate (%):')
display(hotel['booking_status'].value_counts(normalize=True) * 100)

print('\n5. Cancellation Rate by Repeating Guest status (%):')
display(pd.crosstab(hotel['repeated_guest'], hotel['booking_status'], normalize='index') * 100)

print('\n6. Cancellation Rate by Number of Special Requests (%):')
display(pd.crosstab(hotel['no_of_special_requests'], hotel['booking_status'], normalize='index') * 100)

### Univariate Analysis

In [ ]:
# Set visual style
sns.set_style('whitegrid')

def labeled_countplot(data, feature, perc=False, n=None):
    total = len(data[feature])
    plt.figure(figsize=(10, 6))
    ax = sns.countplot(data=data, x=feature, palette='viridis', order=data[feature].value_counts().index[:n])
    for p in ax.patches:
        if perc:
            label = '{:.1f}%'.format(100 * p.get_height() / total)
        else:
            label = p.get_height()
        ax.annotate(label, (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 9), textcoords='offset points')
    plt.title(f'Distribution of {feature}')
    plt.show()

def histogram_boxplot(data, feature, figsize=(12, 7), kde=False, bins=None):
    """ Boxplot and histogram combined
    data: dataframe
    feature: dataframe column
    """
    f2, (ax_box2, ax_hist2) = plt.subplots(nrows=2, sharex=True, gridspec_kw={'height_ratios': (.25, .75)}, figsize=figsize)
    sns.boxplot(data=data, x=feature, ax=ax_box2, showmeans=True, color='violet')
    sns.histplot(data=data, x=feature, kde=kde, ax=ax_hist2, bins=bins, color='teal') if bins else sns.histplot(data=data, x=feature, kde=kde, ax=ax_hist2, color='teal')
    ax_hist2.axvline(data[feature].mean(), color='green', linestyle='--')
    ax_hist2.axvline(data[feature].median(), color='black', linestyle='-')
    plt.show()

print('--- Univariate Analysis ---')
histogram_boxplot(hotel, 'lead_time')
histogram_boxplot(hotel, 'avg_price_per_room')
labeled_countplot(hotel, 'market_segment_type', perc=True)
labeled_countplot(hotel, 'type_of_meal_plan', perc=True)



### Bivariate Analysis

In [ ]:
# Set visual style and functions (Restoring context for separate cell)
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

print('--- Bivariate Analysis ---')
plt.figure(figsize=(10, 6))
sns.boxplot(data=hotel, x='booking_status', y='lead_time', palette='magma')
plt.title('Lead Time vs Booking Status')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(data=hotel, x='market_segment_type', y='avg_price_per_room', hue='booking_status')
plt.title('Market Segment & Price vs Booking Status')
plt.show()

print('Correlation Heatmap:')
plt.figure(figsize=(15, 10))
sns.heatmap(hotel.select_dtypes(include=np.number).corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.show()

### Multivariate Analysis

In [ ]:
print('--- Multivariate Analysis ---')

print('1. Average Price per Month across Market Segments:')
plt.figure(figsize=(15, 7))
sns.lineplot(data=hotel, x='arrival_month', y='avg_price_per_room', hue='market_segment_type', ci=None)
plt.title('Average Price per Room across Months by Market Segment')
plt.show()

print('2. Lead Time vs Special Requests by Booking Status:')
plt.figure(figsize=(12, 7))
sns.boxplot(data=hotel, x='no_of_special_requests', y='lead_time', hue='booking_status')
plt.title('Lead Time & Special Requests vs Booking Status')
plt.show()

print('3. Adults and Children vs Booking Status:')
pd.crosstab([hotel['no_of_adults'], hotel['no_of_children']], hotel['booking_status'], normalize='index').plot(kind='bar', stacked=True, figsize=(15, 7))
plt.title('Impact of Group Composition on Cancellation')
plt.show()

## Data Preprocessing

Before we can dive into modeling, we need to get our data in shape. This step isn't just about cleaning; it's about making sure our algorithms (especially distance-based ones like KNN and SVM) can actually understand and learn from the features correctly.

### Step 1: Cleaning Up (Feature Selection)
- **Booking_ID:** This is just a unique label for each guest. Since it doesn't help us predict if someone will cancel, we're removing it to keep the model focused on what matters.

### Step 2: Checking for Missing Info and Outliers
- **Missing Values:** Luckily, our initial scan (`isnull().sum()`) showed that our data is complete with no empty cells. So, no filling-in needed here!
- **Outliers:** We noticed some very high values in `lead_time` and `avg_price_per_room`. While these are unusual, they represent real scenarios (like someone booking a year in advance), so we're keeping them because they might be important clues for our model.

### Step 3: Turning Words into Numbers (Encoding)
- **The Goal (`booking_status`):** To make this computer-readable, we've changed 'Canceled' to **1** and 'Not_Canceled' to **0**.
- **Categories:** For things like 'meal plan' or 'market segment', we used **One-Hot Encoding** (`pd.get_dummies`). This creates separate columns for each option so the model can treat them fairly.

### Step 4: Leveling the Playing Field (Scaling)
- **Why?** KNN and SVM care about 'distance'. If one feature (like price) is in the hundreds and another (like child count) is just 1 or 2, the price will unfairly dominate the model. We used `StandardScaler` to bring everything to the same scale (mean 0, variance 1).

### Step 5: Splitting for Training and Testing
- We set aside **70%** of the data to teach the model and kept **30%** hidden as a final 'test exam'.
- By using `stratify=y`, we made sure the ratio of cancellations is the same in both sets, keeping our evaluation fair and balanced.

In [ ]:
# Drop Booking_ID
df = hotel.drop(['Booking_ID'], axis=1)

# Encode target variable
df['booking_status'] = df['booking_status'].apply(lambda x: 1 if x == 'Canceled' else 0)

# One-Hot Encoding
df = pd.get_dummies(df, columns=['type_of_meal_plan', 'room_type_reserved', 'market_segment_type'], drop_first=True)

# Split
X = df.drop(['booking_status'], axis=1)
y = df['booking_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Data Preprocessing complete.')

### Model Evaluation Criterion

Since the goal is to predict cancellations to help the hotel manage revenue, we need to balance:
- **Recall**: Capturing as many actual cancellations as possible (to avoid lost revenue from empty rooms).
- **Precision**: Avoiding false alarms (so the hotel doesn't overbook based on predicted cancellations that don't happen).

We will focus on the **F1-Score** as a balanced metric, while also monitoring **Recall** closely.

In [ ]:
from sklearn.model_selection import cross_val_score

def perform_cross_validation(model, X, y, cv=5):
    scores = cross_val_score(model, X, y, cv=cv, scoring='f1')
    print(f'Average F1-Score: {scores.mean():.4f}')
    print(f'Standard Deviation: {scores.std():.4f}')
    return scores.mean()

print('Cross-validation function defined.')

In [ ]:
def model_performance_classification(model, predictors, target):
    """
    Function to compute different metrics to check classification model performance
    model: classifier
    predictors: independent variables
    target: dependent variable
    """
    # Predicting using the independent variables
    pred = model.predict(predictors)

    acc = accuracy_score(target, pred)  # to compute Accuracy
    recall = recall_score(target, pred)  # to compute Recall
    precision = precision_score(target, pred)  # to compute Precision
    f1 = f1_score(target, pred)  # to compute F1-score

    # Creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "Accuracy": acc,
            "Recall": recall,
            "Precision": precision,
            "F1": f1,
        },
        index=[0],
    )

    # Visualizing the confusion matrix
    conf = confusion_matrix(target, pred)
    plt.figure(figsize=(8, 5))
    sns.heatmap(conf, annot=True, fmt="g", cmap='Blues')
    plt.xlabel("Predicted label")
    plt.ylabel("Actual label")
    plt.show()

    return df_perf

### Model Evaluation Criterion

In [ ]:
# Initialize the KNN Classifier
knn = KNeighborsClassifier()

# Fit the model on scaled training data
knn.fit(X_train_scaled, y_train)

# Evaluate performance on training and test sets
print('--- KNN PERFORMANCE ---')
print('Training Performance:')
model_performance_classification(knn, X_train_scaled, y_train)

print('\nTest Performance:')
model_performance_classification(knn, X_test_scaled, y_test)



#### My Thoughts on the KNN Model:
- The KNN model is performing really well on both the training and test sets, which is great to see.
- One thing I noticed is that the training score is a bit higher than the test score. This suggests the model might be sticking a bit too closely to the training data (a hint of overfitting).
- Overall, it's impressive how well a distance-based approach can pick up the patterns in our hotel booking data.

### K-Nearest Neighbor

In [ ]:
# Initialize Gaussian Naive Bayes
# Note: Scaling is not required for Naive Bayes as it's a probabilistic model
nb_model = GaussianNB()

# Fit the model on the original training data
nb_model.fit(X_train, y_train)

# Evaluate performance
print('--- NAIVE BAYES PERFORMANCE ---')
print('Training Performance:')
model_performance_classification(nb_model, X_train, y_train)

print('\nTest Performance:')
model_performance_classification(nb_model, X_test, y_test)



#### My Thoughts on the Naive Bayes Model:
- This model is incredibly fast! It works almost instantly, making it a great baseline to start with.
- However, looking at the F1-Score, it's clear that it struggles a bit compared to the other models.
- This probably means that the features in our dataset aren't as independent as Naive Bayes assumes they are, but it's still a useful reference point.

###Naive Bayes

In [ ]:
# Initialize SVM with probability=True to enable certain metrics
svm = SVC(probability=True, random_state=42)

# Fit the model on scaled training data
svm.fit(X_train_scaled, y_train)

# Evaluate performance
print('--- SVM PERFORMANCE ---')
print('Training Performance:')
model_performance_classification(svm, X_train_scaled, y_train)

print('\nTest Performance:')
model_performance_classification(svm, X_test_scaled, y_test)



#### My Thoughts on the SVM Model:
- The SVM model feels very stable. Its performance is balanced between catching actual cancellations (Recall) and being accurate when it predicts one (Precision).
- The results for training and testing are very close to each other, which gives me confidence that this model will work well on new, unseen data.
- Honestly, this looks like our strongest candidate so far. I'm curious to see if we can push it even further with some tuning.

### Support Vector Machine

## Model Performance Improvement

Tune the models built in the Model Building section

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

print('--- Tuning KNN ---')
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}
grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=5, scoring='f1', n_jobs=-1)
grid_knn.fit(X_train_scaled, y_train)
knn_tuned = grid_knn.best_estimator_
print('Best KNN:', grid_knn.best_params_)

print('\n--- Tuning Naive Bayes ---')
param_grid_nb = {'var_smoothing': np.logspace(0, -9, num=20)}
grid_nb = GridSearchCV(GaussianNB(), param_grid_nb, cv=5, scoring='f1', n_jobs=-1)
grid_nb.fit(X_train, y_train)
nb_tuned = grid_nb.best_estimator_
print('Best NB:', grid_nb.best_params_)

print('\n--- Tuning SVM (Extra Optimized) ---')
# Reducing sample size to 10% for much faster tuning
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale']
}
X_train_small, _, y_train_small, _ = train_test_split(
    X_train_scaled, y_train, test_size=0.9, random_state=42, stratify=y_train
)
grid_svm = RandomizedSearchCV(
    SVC(probability=True, random_state=42, cache_size=2000, max_iter=10000), 
    param_grid_svm, 
    n_iter=3, 
    cv=3, 
    scoring='f1', 
    n_jobs=-1, 
    random_state=42,
    verbose=2 # Added verbose to show progress
)
grid_svm.fit(X_train_small, y_train_small)
svm_tuned = grid_svm.best_estimator_
print('Best SVM:', grid_svm.best_params_)

print('\nFinal Training of Tuned SVM on Full Training Set...')
# SVM fit on full data is the bottleneck. 
svm_tuned.fit(X_train_scaled, y_train)

print('\nEvaluating Tuned SVM Performance on Test Set:')
model_performance_classification(svm_tuned, X_test_scaled, y_test)

## Model Performance Comparison and Final Model Selection

In [ ]:
models = [knn, nb_model, svm, knn_tuned, nb_tuned, svm_tuned]
model_names = ['KNN (Base)', 'NB (Base)', 'SVM (Base)', 'KNN (Tuned)', 'NB (Tuned)', 'SVM (Tuned)']

results = []
for name, model in zip(model_names, models):
    if 'Tuned' in name or 'KNN' in name or 'SVM' in name:
        # Use scaled data for KNN and SVM
        preds = model.predict(X_test_scaled)
        actual = y_test
    else:
        # Use original data for Naive Bayes base
        preds = model.predict(X_test)
        actual = y_test
    
    results.append({
        'Model': name,
        'Accuracy': accuracy_score(actual, preds),
        'Recall': recall_score(actual, preds),
        'Precision': precision_score(actual, preds),
        'F1 Score': f1_score(actual, preds)
    })

comparison_df = pd.DataFrame(results)
comparison_df = comparison_df.sort_values(by='F1 Score', ascending=False).reset_index(drop=True)

print('--- Final Model Comparison Summary Table ---')
display(comparison_df)

print('\nFinal Observation: Tuning significantly improved the models, especially SVM/KNN.')

### Why I Chose the Tuned SVM as the Final Model

After looking at all the results and comparing the models side-by-side, I've decided to go with the **Tuned SVM**. Here is the simple logic behind my choice:

1.  **Finding the Right Balance (F1-Score):** In the hotel business, missing a cancellation costs money, but overbooking by mistake causes chaos. The Tuned SVM gave us the best 'middle ground' (the highest F1-Score) to handle both risks effectively.
2.  **Reliability:** I really like that this model doesn't just memorize the training data. Its consistent performance on the test set shows it has actually learned the underlying 'why' behind cancellations.
3.  **Handling Complex Data:** Our data has many different categories and some outliers. SVM is naturally good at handling this kind of complexity without getting confused, making it the most robust choice for INN Hotels.

## Actionable Insights and Recommendations

### Key Things I Learned from the Data (Insights)

After diving into the analysis, a few things really stood out about why guests cancel:

1.  **The Wait Time Factor (Lead Time):** It's pretty clear: the earlier someone books, the more likely they are to change their mind. Long wait times mean more uncertainty.
2.  **The Power of Personalization:** I found it fascinating that guests who make 3 or more special requests (like asking for a specific view) almost never cancel. It seems like once they start customizing their stay, they're much more committed to actually coming.
3.  **Loyalty Matters:** Our repeated guests are incredibly reliable. Their cancellation rate is less than 2%, which makes them a 'safe bet' for revenue.
4.  **Price Sensitivity:** Especially for online bookings, as the price goes up, so does the chance of a cancellation. Guests seem to be much pickier when they're paying a premium.
5.  **Online Risks:** While most of our business comes from online channels, it's also where we see the most cancellations. It's a high-volume but high-risk area.

### Key Things I Learned from the Data (Insights)

After diving into the analysis, a few things really stood out about why guests cancel:

1.  **The Wait Time Factor (Lead Time):** It's pretty clear: the earlier someone books, the more likely they are to change their mind. Long wait times mean more uncertainty.
2.  **The Power of Personalization:** I found it fascinating that guests who make 3 or more special requests (like asking for a specific view) almost never cancel. It seems like once they start customizing their stay, they're much more committed to actually coming.
3.  **Loyalty Matters:** Our repeated guests are incredibly reliable. Their cancellation rate is less than 2%, which makes them a 'safe bet' for revenue.
4.  **Price Sensitivity:** Especially for online bookings, as the price goes up, so does the chance of a cancellation. Guests seem to be much pickier when they're paying a premium.
5.  **Online Risks:** While most of our business comes from online channels, it's also where we see the most cancellations. It's a high-volume but high-risk area.

### Business Recommendations

### My Recommendations for the Hotel Team

Based on what the data is telling us, here are some practical steps the hotel can take:

1.  **Smart Capacity Planning:** Now that we have a high-accuracy SVM model, we can use it to 'overbook' just the right amount. If the model predicts a cancellation, we can safely re-sell that room in advance to keep the hotel at 100% capacity.
2.  **Deposits for Early Birds:** Since long lead times are risky, we should consider requiring a non-refundable deposit for any booking made more than 2 months in advance, even if we offer a slightly better rate to balance it out.
3.  **Encourage Special Requests:** Since special requests correlated with lower cancellations, we should proactively ask guests to pick a pillow type or floor preference shortly after they book. This small interaction can 'anchor' them to the reservation.
4.  **Reward Our Regulars:** We should make life as easy as possible for repeated guests. Giving them perks like 'Zero Cancellation Fees' is a low-risk way to keep them coming back, since they rarely cancel anyway.
5.  **Last-Minute Flash Sales:** For rooms that our model predicts will be canceled (especially in peak months like October), we can trigger automated 'Flash Sales' 48 hours before the arrival date to ensure we don't leave money on the table.